In [1]:
import pandas as pd
import numpy as np
import os
import re
from functools import reduce

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

In [5]:
path ='./baseball'

In [6]:
file_list = os.listdir(path)

In [15]:
file_list.sort()
file_list

['선수정보_2018.csv', '선수정보_2019.csv', '선수정보_2020.csv', '선수정보_2021.csv']

In [120]:
baseball=[]
for ls in file_list[:4]:
    baseball.append(pd.read_csv(os.path.join(path,ls), encoding='utf-8'))
    

In [121]:
baseball[0]['year']=file_list[0][-5:3]

In [122]:
for i in range(len(file_list)):
    baseball[i]['year']= int(file_list[i][5:9])

In [123]:
baseball[3]

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA,year
0,라모스,04-04,@NC,W,2:1,2,1B,1,3,0,...,0,0.000,0.250,0.000,0.250,15,2.04,-1.12,-0.092,2021
1,라모스,04-07,@KT,L,3:7,4,1B,1,2,0,...,0,0.111,0.200,0.111,0.311,6,1.85,-1.07,-0.093,2021
2,라모스,04-10,SSG,L,3:4,2,1B,1,4,2,...,0,0.286,0.375,0.571,0.946,27,1.89,1.94,0.280,2021
3,라모스,04-13,@키움,L,2:8,2,1B,1,4,0,...,0,0.190,0.261,0.381,0.642,18,0.50,-1.03,-0.052,2021
4,라모스,04-15,@키움,W,6:4,4,1B,1,4,1,...,0,0.241,0.313,0.379,0.692,16,1.17,-1.04,-0.085,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10428,김상수,08-27,@LG,L,1:3,9,2B,1,3,0,...,0,0.221,0.310,0.264,0.575,13,1.14,-0.03,0.031,2021
10429,김상수,08-29,@KT,L,3:8,9,2B,1,3,0,...,0,0.223,0.312,0.265,0.577,18,0.50,0.66,0.046,2021
10430,김상수,09-01,키움,W,7:1,9,2B,1,2,2,...,0,0.227,0.320,0.268,0.588,18,0.67,2.07,0.098,2021
10431,김상수,09-03,@KIA,W,9:3,6,2B,1,3,1,...,0,0.228,0.322,0.268,0.590,11,0.57,0.90,0.060,2021


In [124]:
tot_baseball = reduce(lambda left, right: pd.concat([left,right]), baseball)

In [125]:
tot_baseball=tot_baseball.reset_index(drop=True)

In [126]:
tot_baseball['상대']=tot_baseball['상대'].str.replace(pat=r'[^\w]',repl='', regex=True)

In [117]:
tot_baseball['상대'].value_counts()

두산      3304
@삼성     3178
@LG     3178
NC      3173
@한화     3172
@KT     3169
삼성      3163
@KIA    3152
한화      3149
롯데      3135
@두산     3120
@NC     3119
KIA     3115
LG      3103
@롯데     3088
KT      3032
@SK     2669
SK      2646
키움      2401
@키움     2309
넥센       917
@넥센      907
@SSG     548
SSG      513
Name: 상대, dtype: int64

In [127]:
tot_baseball['상대'].value_counts()

두산     6424
삼성     6341
한화     6321
NC     6292
LG     6281
KIA    6267
롯데     6223
KT     6201
SK     5315
키움     4710
넥센     1824
SSG    1061
Name: 상대, dtype: int64

In [11]:
tot_baseball = pd.read_csv('total_baseball.csv')

In [132]:
WINDOW_SIZE = 20

In [131]:
def make_time_series(df,window_size):
    
    barell = ['2타','3타','홈런']
    on_base = ['안타','2타','3타','홈런','사구','볼넷','고4']
    out = ['병살','삼진','희타']
    etc = ['타수','도루','투구']
    is_bin = ['선발']


    # for calcaul
    total_base =['타수','볼넷','고4','사구','희타']
    hit = ['안타','2타','3타','홈런']
    
    sample_list = []
    
    for i in range(1,len(df),window_size):
        single = df[-i:-(i+window_size):-1].groupby(['이름','year'])[on_base+out+etc].sum()
        single['num_game'] = df[-i:-(i+window_size):-1].shape[0]
        single['on_base']= single[on_base].sum(axis=1)/single[total_base].sum(axis=1)
        single['slg'] = (single[hit] * [1,2,3,4]).sum(axis=1)/single['타수']
        sample_list.append(single)
    
    result = pd.concat(sample_list).reset_index()
    result.index = ['t'+str(i) for i in range(1,len(result)+1)]
    return result
    

In [125]:
base_2021=tot_baseball[(tot_baseball['year']==2021)]

In [135]:
total_time_seires = []

In [136]:
players= base_2021['이름'].value_counts().keys().to_list()
for player in players:
    sample_df = tot_baseball[(tot_baseball['year']==2021)&(tot_baseball['이름']==player)]
    total_time_seires.append(make_time_series(sample_df,WINDOW_SIZE))

In [141]:
temp = pd.concat(total_time_seires)

In [142]:
temp

,이름,year,안타,2타,3타,홈런,사구,볼넷,고4,병살,삼진,희타,타수,도루,투구,num_game,on_base,slg
t1,피렐라,2021,20,3,0,4,1,5,0,2,13,0,85,1,352,20,0.362637,0.494118
t2,피렐라,2021,30,4,0,4,2,9,1,0,11,0,80,3,336,20,0.543478,0.675000
t3,피렐라,2021,21,1,1,6,0,12,1,1,13,0,74,1,321,20,0.482759,0.675676
t4,피렐라,2021,23,5,0,4,1,10,0,0,14,0,83,2,380,20,0.457447,0.590361
t5,피렐라,2021,25,4,1,5,1,3,0,1,8,0,78,2,316,19,0.475610,0.717949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t1,민병헌,2021,8,3,0,0,0,6,0,2,11,1,42,0,183,14,0.346939,0.333333
t1,이해창,2021,9,3,0,0,1,1,0,0,9,2,39,0,166,14,0.325581,0.384615
t1,이진영,2021,8,2,1,2,4,3,0,0,19,0,35,1,189,13,0.476190,0.657143
t1,김수환,2021,9,2,0,2,0,2,0,2,17,0,41,0,172,13,0.348837,0.512195
